In [ ]:
# PMFlow BNN v0.2.0 Comprehensive Testing Notebook
# ================================================
# 
# This notebook thoroughly tests the PMFlow BNN v0.2.0 library with:
# - Automatic hardware detection and configuration
# - CPU core / GPU core utilization monitoring  
# - Performance benchmarks for multi-system comparison
# - Complete feature validation
# - Detailed metrics collection

print("🎯 PMFlow BNN v0.2.0 Comprehensive Testing Notebook")
print("="*55)
print("Loading testing framework...")

# 🖥️ System Detection & Hardware Analysis

This notebook automatically detects hardware capabilities and configures PMFlow BNN accordingly.

In [ ]:
# System Detection and Hardware Analysis
import sys
import os
import platform
import psutil
import time
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Advanced hardware detection
def detect_system_capabilities():
    """Comprehensive system detection with CPU/GPU core counting."""
    
    print("🔍 Detecting System Capabilities...")
    
    # Basic system info
    system_info = {
        'platform': platform.platform(),
        'python_version': sys.version.split()[0],
        'architecture': platform.architecture()[0],
        'processor': platform.processor(),
    }
    
    # CPU detection
    cpu_info = {
        'cpu_count_logical': psutil.cpu_count(logical=True),
        'cpu_count_physical': psutil.cpu_count(logical=False),
        'cpu_freq_max': psutil.cpu_freq().max if psutil.cpu_freq() else 'Unknown',
        'cpu_freq_current': psutil.cpu_freq().current if psutil.cpu_freq() else 'Unknown',
    }
    
    # Memory detection
    memory = psutil.virtual_memory()
    memory_info = {
        'total_ram_gb': memory.total / (1024**3),
        'available_ram_gb': memory.available / (1024**3),
        'ram_usage_percent': memory.percent,
    }
    
    # GPU detection
    gpu_info = {
        'cuda_available': torch.cuda.is_available(),
        'gpu_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
        'gpu_names': [],
        'gpu_memory_gb': [],
        'gpu_compute_capability': []
    }
    
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            gpu_info['gpu_names'].append(props.name)
            gpu_info['gpu_memory_gb'].append(props.total_memory / (1024**3))
            gpu_info['gpu_compute_capability'].append(f"{props.major}.{props.minor}")
    
    # Jetson detection
    jetson_info = detect_jetson_nano()
    
    # Combine all info
    capabilities = {
        'system': system_info,
        'cpu': cpu_info,
        'memory': memory_info,
        'gpu': gpu_info,
        'jetson': jetson_info,
    }
    
    return capabilities

def detect_jetson_nano():
    """Detect if running on Jetson Nano with detailed specs."""
    jetson_info = {
        'is_jetson': False,
        'model': 'Unknown',
        'tegra_version': 'Unknown',
        'cuda_arch': 'Unknown'
    }
    
    try:
        # Check device tree model
        with open('/proc/device-tree/model', 'r') as f:
            model = f.read().strip()
            if 'jetson' in model.lower() or 'tegra' in model.lower():
                jetson_info['is_jetson'] = True
                jetson_info['model'] = model
        
        # Check Tegra version
        if os.path.exists('/proc/device-tree/compatible'):
            with open('/proc/device-tree/compatible', 'r') as f:
                compatible = f.read()
                if 'tegra210' in compatible:
                    jetson_info['tegra_version'] = 'Tegra X1 (210)'
                    jetson_info['cuda_arch'] = 'Maxwell (5.3)'
    except:
        pass
    
    return jetson_info

def print_system_summary(capabilities):
    """Print formatted system summary."""
    
    print(f"\n📋 SYSTEM SUMMARY:")
    print(f"   Platform: {capabilities['system']['platform']}")
    print(f"   Architecture: {capabilities['system']['architecture']}")
    print(f"   Python: {capabilities['system']['python_version']}")
    
    print(f"\n🖥️  CPU INFORMATION:")
    print(f"   Logical Cores: {capabilities['cpu']['cpu_count_logical']}")
    print(f"   Physical Cores: {capabilities['cpu']['cpu_count_physical']}")
    print(f"   Max Frequency: {capabilities['cpu']['cpu_freq_max']} MHz")
    
    print(f"\n💾 MEMORY INFORMATION:")
    print(f"   Total RAM: {capabilities['memory']['total_ram_gb']:.1f} GB")
    print(f"   Available RAM: {capabilities['memory']['available_ram_gb']:.1f} GB")
    print(f"   Usage: {capabilities['memory']['ram_usage_percent']:.1f}%")
    
    print(f"\n🎮 GPU INFORMATION:")
    if capabilities['gpu']['cuda_available']:
        print(f"   CUDA Available: Yes")
        print(f"   GPU Count: {capabilities['gpu']['gpu_count']}")
        for i, (name, memory, compute) in enumerate(zip(
            capabilities['gpu']['gpu_names'],
            capabilities['gpu']['gpu_memory_gb'],
            capabilities['gpu']['gpu_compute_capability']
        )):
            print(f"   GPU {i}: {name}")
            print(f"     Memory: {memory:.1f} GB")
            print(f"     Compute: {compute}")
    else:
        print(f"   CUDA Available: No")
    
    if capabilities['jetson']['is_jetson']:
        print(f"\n🤖 JETSON INFORMATION:")
        print(f"   Model: {capabilities['jetson']['model']}")
        print(f"   Tegra: {capabilities['jetson']['tegra_version']}")
        print(f"   CUDA Arch: {capabilities['jetson']['cuda_arch']}")

# Run system detection
system_capabilities = detect_system_capabilities()
print_system_summary(system_capabilities)

# Determine optimal device
if system_capabilities['gpu']['cuda_available']:
    device = torch.device('cuda')
    print(f"\n✅ Using GPU: {system_capabilities['gpu']['gpu_names'][0]}")
else:
    device = torch.device('cpu')
    print(f"\n✅ Using CPU: {system_capabilities['cpu']['cpu_count_logical']} logical cores")

print(f"\n🎯 System detection complete!")

# 📚 Library Import & Configuration

Import PMFlow BNN library and configure based on detected hardware.

In [ ]:
# Library Import and Configuration
import sys
import os

# Method 1: Try GitHub installation (preferred after library is pushed)
print("🚀 Attempting to install PMFlow BNN v0.2.0 from GitHub...")
print("📦 Installing: pip install git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2")

try:
    import subprocess
    
    # Install from GitHub - Python 3.6+ compatible subprocess call
    result = subprocess.run([
        sys.executable, '-m', 'pip', 'install', 
        'git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2'
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    
    if result.returncode == 0:
        print("✅ Successfully installed PMFlow BNN v0.2.0 from GitHub!")
        installation_source = "github"
    else:
        print(f"⚠️ GitHub installation failed: {result.stderr}")
        print("📂 Falling back to local development path...")
        installation_source = "local"
except Exception as e:
    print(f"⚠️ GitHub installation error: {e}")
    print("📂 Falling back to local development path...")
    installation_source = "local"

# Method 2: Local development fallback
if installation_source == "local":
    pmflow_library_path = '/home/tmumford/Documents/gravity/programs/demos/machine_learning/nn_lib_v2'
    if os.path.exists(pmflow_library_path) and pmflow_library_path not in sys.path:
        sys.path.insert(0, pmflow_library_path)
        print(f"📂 Added local library path: {pmflow_library_path}")

# Try to import PMFlow BNN
try:
    from pmflow_bnn import (
        get_model_v2, 
        get_performance_config, 
        PMFlowEvaluator,
        benchmark_temporal_parallelism,
        validate_embarrassingly_parallel_scaling
    )
    print("✅ PMFlow BNN library imported successfully")
    library_available = True
    
    # Get version info
    try:
        from pmflow_bnn.version import __version__
        print(f"📦 PMFlow BNN version: {__version__}")
    except:
        print("📦 PMFlow BNN version: Development")
    
    print(f"📍 Installation source: {installation_source}")
        
except ImportError as e:
    print(f"❌ PMFlow BNN library not available: {e}")
    print("📝 Note: This notebook can run system detection but requires library for testing")
    library_available = False
    installation_source = "none"

# Configure based on detected hardware
if library_available:
    print(f"\n🔧 Configuring PMFlow BNN for detected hardware...")
    
    # Determine hardware profile
    if system_capabilities['jetson']['is_jetson']:
        hardware_profile = 'jetson_nano'
        print(f"   Hardware profile: Jetson Nano")
    elif system_capabilities['gpu']['gpu_count'] > 1:
        hardware_profile = 'multi_gpu'
        print(f"   Hardware profile: Multi-GPU ({system_capabilities['gpu']['gpu_count']} GPUs)")
    elif system_capabilities['gpu']['cuda_available']:
        hardware_profile = 'single_gpu'
        print(f"   Hardware profile: Single GPU")
    else:
        hardware_profile = 'cpu'
        print(f"   Hardware profile: CPU ({system_capabilities['cpu']['cpu_count_logical']} cores)")
    
    # Get optimized configuration
    pmflow_config = get_performance_config(hardware_profile)
    print(f"   Model type: {pmflow_config['model_type']}")
    print(f"   Centers: {pmflow_config['n_centers']}")
    print(f"   PM steps: {pmflow_config['pm_steps']}")
    print(f"   Temporal stages: {pmflow_config['temporal_stages']}")
    
    print(f"✅ PMFlow BNN configured for optimal performance")
    
    # Display installation instructions for other environments
    print(f"\n💡 USAGE IN OTHER ENVIRONMENTS:")
    print(f"   Google Colab:")
    print(f"   !pip install git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2")
    print(f"   ")
    print(f"   Jetson Nano:")
    print(f"   pip install git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2")
    print(f"   ")
    print(f"   Any Python environment:")
    print(f"   pip install git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2")

else:
    print(f"⏳ PMFlow BNN configuration pending library availability")
    print(f"\n🚀 TO USE THIS NOTEBOOK:")
    print(f"   1. Ensure PMFlow BNN v0.2.0 is pushed to GitHub")
    print(f"   2. Run: pip install git+https://github.com/experimentech/Pushing-Medium.git#subdirectory=programs/demos/machine_learning/nn_lib_v2")
    print(f"   3. Restart notebook and re-run cells")

# ⚡ Performance Monitoring & Resource Utilization

Real-time monitoring of CPU cores, GPU cores, and system resources during PMFlow execution.

In [ ]:
# Performance Monitoring and Resource Utilization
import threading
import time
from collections import defaultdict
import matplotlib.pyplot as plt

class SystemMonitor:
    """Real-time system resource monitoring during PMFlow execution."""
    
    def __init__(self, device):
        self.device = device
        self.monitoring = False
        self.data = defaultdict(list)
        self.timestamps = []
        
    def start_monitoring(self):
        """Start background monitoring thread."""
        if self.monitoring:
            return
            
        self.monitoring = True
        self.data.clear()
        self.timestamps.clear()
        
        def monitor_loop():
            start_time = time.time()
            
            while self.monitoring:
                current_time = time.time() - start_time
                self.timestamps.append(current_time)
                
                # CPU monitoring
                cpu_percent = psutil.cpu_percent(interval=None, percpu=True)
                self.data['cpu_total'].append(psutil.cpu_percent(interval=None))
                for i, percent in enumerate(cpu_percent):
                    self.data[f'cpu_core_{i}'].append(percent)
                
                # Memory monitoring
                memory = psutil.virtual_memory()
                self.data['memory_used_gb'].append(memory.used / (1024**3))
                self.data['memory_percent'].append(memory.percent)
                
                # GPU monitoring (if available)
                if self.device.type == 'cuda':
                    try:
                        # GPU memory
                        gpu_memory_allocated = torch.cuda.memory_allocated(self.device) / (1024**3)
                        gpu_memory_reserved = torch.cuda.memory_reserved(self.device) / (1024**3)
                        self.data['gpu_memory_allocated_gb'].append(gpu_memory_allocated)
                        self.data['gpu_memory_reserved_gb'].append(gpu_memory_reserved)
                        
                        # GPU utilization (approximate via memory usage)
                        total_gpu_memory = torch.cuda.get_device_properties(self.device).total_memory / (1024**3)
                        gpu_utilization = (gpu_memory_allocated / total_gpu_memory) * 100
                        self.data['gpu_utilization_percent'].append(gpu_utilization)
                        
                    except Exception as e:
                        self.data['gpu_memory_allocated_gb'].append(0)
                        self.data['gpu_memory_reserved_gb'].append(0)
                        self.data['gpu_utilization_percent'].append(0)
                
                time.sleep(0.1)  # Monitor every 100ms
        
        self.monitor_thread = threading.Thread(target=monitor_loop, daemon=True)
        self.monitor_thread.start()
        
    def stop_monitoring(self):
        """Stop monitoring and return collected data."""
        self.monitoring = False
        if hasattr(self, 'monitor_thread'):
            self.monitor_thread.join(timeout=1.0)
        
        return dict(self.data), self.timestamps.copy()
    
    def get_summary_stats(self):
        """Get summary statistics from monitoring data."""
        if not self.data:
            return {}
        
        stats = {}
        
        # CPU stats
        if 'cpu_total' in self.data:
            stats['cpu_avg_percent'] = np.mean(self.data['cpu_total'])
            stats['cpu_max_percent'] = np.max(self.data['cpu_total'])
            stats['cpu_cores_used'] = sum(1 for core in range(system_capabilities['cpu']['cpu_count_logical']) 
                                         if f'cpu_core_{core}' in self.data and 
                                         np.mean(self.data[f'cpu_core_{core}']) > 5.0)
        
        # Memory stats
        if 'memory_percent' in self.data:
            stats['memory_avg_percent'] = np.mean(self.data['memory_percent'])
            stats['memory_max_gb'] = np.max(self.data['memory_used_gb'])
        
        # GPU stats
        if 'gpu_utilization_percent' in self.data:
            stats['gpu_avg_percent'] = np.mean(self.data['gpu_utilization_percent'])
            stats['gpu_max_memory_gb'] = np.max(self.data['gpu_memory_allocated_gb'])
        
        return stats

def create_utilization_plots(monitor_data, timestamps, title="Resource Utilization"):
    """Create comprehensive resource utilization plots."""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{title} - Real-time Resource Monitoring', fontsize=14, fontweight='bold')
    
    # CPU utilization
    ax1 = axes[0, 0]
    if 'cpu_total' in monitor_data:
        ax1.plot(timestamps, monitor_data['cpu_total'], 'b-', linewidth=2, label='Total CPU')
        
        # Plot individual cores (up to 8 for readability)
        colors = plt.cm.Set3(np.linspace(0, 1, min(8, system_capabilities['cpu']['cpu_count_logical'])))
        for i in range(min(8, system_capabilities['cpu']['cpu_count_logical'])):
            if f'cpu_core_{i}' in monitor_data:
                ax1.plot(timestamps, monitor_data[f'cpu_core_{i}'], 
                        color=colors[i], alpha=0.7, linewidth=1, label=f'Core {i}')
    
    ax1.set_xlabel('Time (seconds)')
    ax1.set_ylabel('CPU Usage (%)')
    ax1.set_title('CPU Core Utilization')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 100)
    
    # Memory utilization
    ax2 = axes[0, 1]
    if 'memory_percent' in monitor_data:
        ax2.plot(timestamps, monitor_data['memory_percent'], 'g-', linewidth=2, label='Memory %')
        ax2_gb = ax2.twinx()
        ax2_gb.plot(timestamps, monitor_data['memory_used_gb'], 'g--', linewidth=2, alpha=0.7, label='Memory GB')
        ax2_gb.set_ylabel('Memory Usage (GB)', color='g')
    
    ax2.set_xlabel('Time (seconds)')
    ax2.set_ylabel('Memory Usage (%)', color='g')
    ax2.set_title('Memory Utilization')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 100)
    
    # GPU utilization
    ax3 = axes[1, 0]
    if 'gpu_utilization_percent' in monitor_data and any(monitor_data['gpu_utilization_percent']):
        ax3.plot(timestamps, monitor_data['gpu_utilization_percent'], 'r-', linewidth=2, label='GPU Utilization')
        ax3_mem = ax3.twinx()
        ax3_mem.plot(timestamps, monitor_data['gpu_memory_allocated_gb'], 'r--', linewidth=2, alpha=0.7, label='GPU Memory')
        ax3_mem.set_ylabel('GPU Memory (GB)', color='r')
        ax3.set_ylabel('GPU Utilization (%)', color='r')
        ax3.set_title('GPU Utilization')
    else:
        ax3.text(0.5, 0.5, 'No GPU Data Available', transform=ax3.transAxes, 
                ha='center', va='center', fontsize=12)
        ax3.set_title('GPU Utilization (N/A)')
    
    ax3.set_xlabel('Time (seconds)')
    ax3.grid(True, alpha=0.3)
    
    # Summary statistics
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    # Calculate and display summary stats
    summary_text = "📊 UTILIZATION SUMMARY:\n\n"
    
    if 'cpu_total' in monitor_data:
        cpu_avg = np.mean(monitor_data['cpu_total'])
        cpu_max = np.max(monitor_data['cpu_total'])
        summary_text += f"🖥️  CPU:\n"
        summary_text += f"   Average: {cpu_avg:.1f}%\n"
        summary_text += f"   Peak: {cpu_max:.1f}%\n"
        
        # Count active cores
        active_cores = sum(1 for i in range(system_capabilities['cpu']['cpu_count_logical'])
                          if f'cpu_core_{i}' in monitor_data and 
                          np.mean(monitor_data[f'cpu_core_{i}']) > 5.0)
        summary_text += f"   Active Cores: {active_cores}/{system_capabilities['cpu']['cpu_count_logical']}\n\n"
    
    if 'memory_percent' in monitor_data:
        mem_avg = np.mean(monitor_data['memory_percent'])
        mem_max_gb = np.max(monitor_data['memory_used_gb'])
        summary_text += f"💾 Memory:\n"
        summary_text += f"   Average: {mem_avg:.1f}%\n"
        summary_text += f"   Peak Usage: {mem_max_gb:.1f} GB\n\n"
    
    if 'gpu_utilization_percent' in monitor_data and any(monitor_data['gpu_utilization_percent']):
        gpu_avg = np.mean(monitor_data['gpu_utilization_percent'])
        gpu_max_mem = np.max(monitor_data['gpu_memory_allocated_gb'])
        summary_text += f"🎮 GPU:\n"
        summary_text += f"   Average: {gpu_avg:.1f}%\n"
        summary_text += f"   Peak Memory: {gpu_max_mem:.1f} GB\n"
    else:
        summary_text += f"🎮 GPU: Not Available\n"
    
    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes, fontsize=10, 
            verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    return fig

# Initialize system monitor
monitor = SystemMonitor(device)
print("✅ System monitor initialized")
print(f"   Monitoring device: {device}")
print(f"   CPU cores available: {system_capabilities['cpu']['cpu_count_logical']}")
if system_capabilities['gpu']['cuda_available']:
    print(f"   GPU memory available: {system_capabilities['gpu']['gpu_memory_gb'][0]:.1f} GB")

print(f"\n📊 Ready for performance monitoring during PMFlow execution")

# 🧪 Comprehensive Feature Testing

Test all PMFlow BNN features with detailed metrics collection for multi-system comparison.

In [ ]:
# Comprehensive Feature Testing
def run_comprehensive_pmflow_tests():
    """Run all PMFlow BNN tests with detailed metrics collection."""
    
    if not library_available:
        print("❌ PMFlow BNN library not available - skipping tests")
        print("📝 Run this notebook after library is available locally or from GitHub")
        return None
    
    print("🧪 Starting Comprehensive PMFlow BNN Testing")
    print("="*50)
    
    # Test results storage
    test_results = {
        'system_info': system_capabilities,
        'hardware_profile': hardware_profile,
        'config': pmflow_config,
        'tests': {}
    }
    
    # Initialize evaluator
    evaluator = PMFlowEvaluator(device=device)
    
    # Test 1: Model Creation and Basic Functionality
    print(f"\n🏗️  Test 1: Model Creation and Basic Functionality")
    try:
        monitor.start_monitoring()
        
        model = get_model_v2(**pmflow_config).to(device)
        param_count = sum(p.numel() for p in model.parameters())
        
        # Test forward pass
        test_input = torch.randn(8, 28*28, device=device)
        with torch.no_grad():
            output = model(test_input)
            if isinstance(output, tuple):
                logits, hidden = output
            else:
                logits = output
        
        monitor_data, timestamps = monitor.stop_monitoring()
        
        test_results['tests']['model_creation'] = {
            'status': 'success',
            'param_count': param_count,
            'output_shape': list(logits.shape),
            'monitor_data': monitor_data,
            'timestamps': timestamps
        }
        
        print(f"   ✅ Model created: {param_count:,} parameters")
        print(f"   ✅ Forward pass: {test_input.shape} → {logits.shape}")
        
    except Exception as e:
        monitor.stop_monitoring()
        test_results['tests']['model_creation'] = {
            'status': 'failed',
            'error': str(e)
        }
        print(f"   ❌ Model creation failed: {e}")
        return test_results
    
    # Test 2: Embarrassingly Parallel Scaling
    print(f"\n🚀 Test 2: Embarrassingly Parallel Scaling")
    try:
        monitor.start_monitoring()
        
        max_batch = 32 if device.type == 'cuda' else 16
        scaling_results = evaluator.evaluate_embarrassingly_parallel_scaling(
            model, max_batch_size=max_batch, input_shape=(28*28,)
        )
        
        monitor_data, timestamps = monitor.stop_monitoring()
        
        test_results['tests']['embarrassingly_parallel'] = {
            'status': 'success',
            'results': scaling_results,
            'monitor_data': monitor_data,
            'timestamps': timestamps
        }
        
        print(f"   ✅ Peak efficiency: {scaling_results['peak_efficiency']:.2f}x")
        print(f"   ✅ Average efficiency: {scaling_results['average_efficiency']:.2f}x")
        print(f"   ✅ Embarrassingly parallel: {scaling_results['is_embarrassingly_parallel']}")
        
    except Exception as e:
        monitor.stop_monitoring()
        test_results['tests']['embarrassingly_parallel'] = {
            'status': 'failed', 
            'error': str(e)
        }
        print(f"   ❌ Scaling test failed: {e}")
    
    # Test 3: Gravitational Center Dynamics
    print(f"\n🌌 Test 3: Gravitational Center Dynamics")
    try:
        monitor.start_monitoring()
        
        # Generate test data
        test_data = torch.randn(100, 28*28)
        test_labels = torch.randint(0, 4, (100,))
        
        dynamics_results = evaluator.evaluate_gravitational_dynamics(
            model, test_data, test_labels, adaptation_steps=10
        )
        
        monitor_data, timestamps = monitor.stop_monitoring()
        
        test_results['tests']['gravitational_dynamics'] = {
            'status': 'success',
            'results': dynamics_results,
            'monitor_data': monitor_data,
            'timestamps': timestamps
        }
        
        if dynamics_results:
            print(f"   ✅ Center movement: {dynamics_results['mean_movement']:.4f}")
            if 'specialization_ratio' in dynamics_results:
                print(f"   ✅ Specialization ratio: {dynamics_results['specialization_ratio']:.2f}x")
        else:
            print(f"   ⚠️ No gravitational centers detected")
        
    except Exception as e:
        monitor.stop_monitoring()
        test_results['tests']['gravitational_dynamics'] = {
            'status': 'failed',
            'error': str(e)
        }
        print(f"   ❌ Gravitational dynamics test failed: {e}")
    
    # Test 4: Biological Plasticity
    print(f"\n🧠 Test 4: Biological Plasticity")
    try:
        monitor.start_monitoring()
        
        # Create shifting datasets for plasticity testing
        train_data = torch.randn(200, 28*28) * 0.8
        train_labels = torch.randint(0, 4, (200,))
        
        shifting_datasets = []
        for i in range(3):
            shift_data = torch.randn(100, 28*28) * (0.8 + i * 0.1)
            shift_labels = torch.randint(0, 4, (100,))
            shifting_datasets.append((shift_data, shift_labels))
        
        plasticity_results = evaluator.evaluate_biological_plasticity(
            model, train_data, train_labels, shifting_datasets
        )
        
        monitor_data, timestamps = monitor.stop_monitoring()
        
        test_results['tests']['biological_plasticity'] = {
            'status': 'success',
            'results': plasticity_results,
            'monitor_data': monitor_data,
            'timestamps': timestamps
        }
        
        print(f"   ✅ Plasticity score: {plasticity_results['plasticity_score']:.3f}")
        print(f"   ✅ Memory retention: {plasticity_results['memory_retention']:.3f}")
        
    except Exception as e:
        monitor.stop_monitoring()
        test_results['tests']['biological_plasticity'] = {
            'status': 'failed',
            'error': str(e)
        }
        print(f"   ❌ Plasticity test failed: {e}")
    
    # Test 5: Performance Benchmarking
    print(f"\n⚡ Test 5: Performance Benchmarking")
    try:
        monitor.start_monitoring()
        
        benchmark_batch_sizes = [4, 8, 16] if device.type == 'cpu' else [8, 16, 32, 64]
        
        benchmark_results = benchmark_temporal_parallelism(
            model, batch_sizes=benchmark_batch_sizes, device=device, num_trials=5
        )
        
        monitor_data, timestamps = monitor.stop_monitoring()
        
        test_results['tests']['performance_benchmark'] = {
            'status': 'success',
            'results': benchmark_results,
            'monitor_data': monitor_data,
            'timestamps': timestamps
        }
        
        print(f"   ✅ Benchmark completed for {len(benchmark_batch_sizes)} batch sizes")
        print(f"   ✅ Throughput range: {min(benchmark_results['throughput']):.1f} - {max(benchmark_results['throughput']):.1f} samples/sec")
        
    except Exception as e:
        monitor.stop_monitoring()
        test_results['tests']['performance_benchmark'] = {
            'status': 'failed',
            'error': str(e)
        }
        print(f"   ❌ Benchmark failed: {e}")
    
    print(f"\n🎉 Comprehensive testing complete!")
    return test_results

# Run tests if library is available
if library_available:
    test_results = run_comprehensive_pmflow_tests()
else:
    print("⏳ Comprehensive testing pending library availability")
    print("📝 This cell will run automatically once the library is imported")
    test_results = None

# 📊 Results Analysis and Reporting

In [ ]:
# Results Analysis and Reporting
def analyze_test_results(test_results):
    """Analyze and visualize comprehensive test results."""
    
    if test_results is None:
        print("📝 No test results available - run comprehensive tests first")
        return
    
    print("📊 PMFlow BNN v0.2.0 Library Test Results")
    print("="*50)
    
    # System Summary
    print(f"\n🖥️  System Configuration:")
    sys_info = test_results['system_info']
    print(f"   Platform: {sys_info['system']['platform']}")
    print(f"   CPU Cores: {sys_info['cpu']['cpu_count_logical']} logical, {sys_info['cpu']['cpu_count_physical']} physical")
    print(f"   Memory: {sys_info['memory']['total_ram_gb']:.1f} GB")
    if sys_info['gpu']['cuda_available']:
        print(f"   GPU: {sys_info['gpu']['gpu_names'][0]} ({sys_info['gpu']['gpu_memory_gb'][0]:.1f} GB)")
    else:
        print(f"   GPU: None (CPU only)")
    print(f"   Profile: {test_results['hardware_profile']}")
    
    # Test Summary
    tests = test_results['tests']
    passed_tests = sum(1 for test in tests.values() if test['status'] == 'success')
    total_tests = len(tests)
    success_rate = (passed_tests / total_tests) * 100 if total_tests > 0 else 0
    
    print(f"\n✅ Test Summary:")
    print(f"   Passed: {passed_tests}/{total_tests} ({success_rate:.1f}%)")
    
    # Individual Test Results
    for test_name, test_data in tests.items():
        status_icon = "✅" if test_data['status'] == 'success' else "❌"
        print(f"   {status_icon} {test_name.replace('_', ' ').title()}")
        
        if test_data['status'] == 'failed':
            print(f"      Error: {test_data['error']}")
    
    # Performance Analysis
    print(f"\n⚡ Performance Analysis:")
    
    # Embarrassingly Parallel Results
    if 'embarrassingly_parallel' in tests and tests['embarrassingly_parallel']['status'] == 'success':
        ep_results = tests['embarrassingly_parallel']['results']
        print(f"   🚀 Embarrassingly Parallel Scaling:")
        print(f"      Peak Efficiency: {ep_results['peak_efficiency']:.2f}x")
        print(f"      Average Efficiency: {ep_results['average_efficiency']:.2f}x")
        print(f"      Is Embarrassingly Parallel: {ep_results['is_embarrassingly_parallel']}")
    
    # Gravitational Dynamics
    if 'gravitational_dynamics' in tests and tests['gravitational_dynamics']['status'] == 'success':
        gd_results = tests['gravitational_dynamics']['results']
        if gd_results:
            print(f"   🌌 Gravitational Dynamics:")
            print(f"      Mean Center Movement: {gd_results['mean_movement']:.4f}")
            if 'specialization_ratio' in gd_results:
                print(f"      Specialization Ratio: {gd_results['specialization_ratio']:.2f}x")
    
    # Biological Plasticity
    if 'biological_plasticity' in tests and tests['biological_plasticity']['status'] == 'success':
        bp_results = tests['biological_plasticity']['results']
        print(f"   🧠 Biological Plasticity:")
        print(f"      Plasticity Score: {bp_results['plasticity_score']:.3f}")
        print(f"      Memory Retention: {bp_results['memory_retention']:.3f}")
    
    # Resource Utilization Summary
    print(f"\n🔧 Resource Utilization:")
    for test_name, test_data in tests.items():
        if test_data['status'] == 'success' and 'monitor_data' in test_data:
            monitor_data = test_data['monitor_data']
            if monitor_data and 'cpu_total' in monitor_data and 'memory_percent' in monitor_data:
                avg_cpu = sum(monitor_data['cpu_total']) / len(monitor_data['cpu_total'])
                avg_memory = sum(monitor_data['memory_percent']) / len(monitor_data['memory_percent'])
                print(f"   {test_name.replace('_', ' ').title()}:")
                print(f"      CPU: {avg_cpu:.1f}% | Memory: {avg_memory:.1f}%")
                
                if 'gpu_utilization_percent' in monitor_data and any(monitor_data['gpu_utilization_percent']):
                    avg_gpu = sum(monitor_data['gpu_utilization_percent']) / len(monitor_data['gpu_utilization_percent'])
                    avg_gpu_mem = sum(monitor_data['gpu_memory_allocated_gb']) / len(monitor_data['gpu_memory_allocated_gb'])
                    print(f"      GPU: {avg_gpu:.1f}% | GPU Memory: {avg_gpu_mem:.1f} GB")
    
    return test_results

def visualize_performance_data(test_results):
    """Create performance visualizations."""
    
    if test_results is None or not any(t['status'] == 'success' for t in test_results['tests'].values()):
        print("📊 No successful test data available for visualization")
        return
    
    try:
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('PMFlow BNN v0.2.0 Performance Analysis', fontsize=16)
        
        # 1. Embarrassingly Parallel Scaling
        if ('embarrassingly_parallel' in test_results['tests'] and 
            test_results['tests']['embarrassingly_parallel']['status'] == 'success'):
            
            ep_results = test_results['tests']['embarrassingly_parallel']['results']
            batch_sizes = ep_results['batch_sizes']
            scaling_efficiency = ep_results['scaling_efficiency']
            
            axes[0,0].plot(batch_sizes, scaling_efficiency, 'b-o', linewidth=2, markersize=6)
            axes[0,0].axhline(y=1.0, color='r', linestyle='--', alpha=0.7, label='Perfect Linear')
            axes[0,0].set_xlabel('Batch Size')
            axes[0,0].set_ylabel('Scaling Efficiency')
            axes[0,0].set_title('Embarrassingly Parallel Scaling')
            axes[0,0].grid(True, alpha=0.3)
            axes[0,0].legend()
            axes[0,0].set_ylim(0, max(scaling_efficiency) * 1.1)
        else:
            axes[0,0].text(0.5, 0.5, 'No Scaling Data Available', transform=axes[0,0].transAxes, 
                          ha='center', va='center', fontsize=12)
            axes[0,0].set_title('Embarrassingly Parallel Scaling (N/A)')
        
        # 2. Resource Utilization Across Tests
        test_names = []
        cpu_usage = []
        memory_usage = []
        
        for test_name, test_data in test_results['tests'].items():
            if test_data['status'] == 'success' and 'monitor_data' in test_data:
                monitor_data = test_data['monitor_data']
                if monitor_data and 'cpu_total' in monitor_data and 'memory_percent' in monitor_data:
                    test_names.append(test_name.replace('_', ' ').title())
                    cpu_usage.append(sum(monitor_data['cpu_total']) / len(monitor_data['cpu_total']))
                    memory_usage.append(sum(monitor_data['memory_percent']) / len(monitor_data['memory_percent']))
        
        if test_names:
            x_pos = range(len(test_names))
            axes[0,1].bar([p - 0.2 for p in x_pos], cpu_usage, 0.4, label='CPU %', alpha=0.8)
            axes[0,1].bar([p + 0.2 for p in x_pos], memory_usage, 0.4, label='Memory %', alpha=0.8)
            axes[0,1].set_xlabel('Test')
            axes[0,1].set_ylabel('Usage %')
            axes[0,1].set_title('Resource Utilization by Test')
            axes[0,1].set_xticks(x_pos)
            axes[0,1].set_xticklabels(test_names, rotation=45, ha='right')
            axes[0,1].legend()
            axes[0,1].grid(True, alpha=0.3)
        else:
            axes[0,1].text(0.5, 0.5, 'No Resource Data Available', transform=axes[0,1].transAxes, 
                          ha='center', va='center', fontsize=12)
            axes[0,1].set_title('Resource Utilization (N/A)')
        
        # 3. Performance Benchmark
        if ('performance_benchmark' in test_results['tests'] and 
            test_results['tests']['performance_benchmark']['status'] == 'success'):
            
            bench_results = test_results['tests']['performance_benchmark']['results']
            batch_sizes = bench_results['batch_sizes']
            throughput = bench_results['throughput']
            
            axes[1,0].plot(batch_sizes, throughput, 'g-s', linewidth=2, markersize=6)
            axes[1,0].set_xlabel('Batch Size')
            axes[1,0].set_ylabel('Throughput (samples/sec)')
            axes[1,0].set_title('Performance Throughput')
            axes[1,0].grid(True, alpha=0.3)
        else:
            axes[1,0].text(0.5, 0.5, 'No Benchmark Data Available', transform=axes[1,0].transAxes, 
                          ha='center', va='center', fontsize=12)
            axes[1,0].set_title('Performance Throughput (N/A)')
        
        # 4. Test Success Summary
        test_statuses = [test['status'] for test in test_results['tests'].values()]
        success_count = test_statuses.count('success')
        failure_count = test_statuses.count('failed')
        
        if success_count + failure_count > 0:
            labels = ['Passed', 'Failed']
            sizes = [success_count, failure_count]
            colors = ['#2ecc71', '#e74c3c']
            
            # Only show pie chart if there are any failures, otherwise show success message
            if failure_count > 0:
                axes[1,1].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
            else:
                axes[1,1].pie([1], labels=['All Tests Passed'], colors=['#2ecc71'], autopct='100.0%%', startangle=90)
            axes[1,1].set_title(f'Test Success Rate ({success_count}/{success_count+failure_count})')
        else:
            axes[1,1].text(0.5, 0.5, 'No Test Data Available', transform=axes[1,1].transAxes, 
                          ha='center', va='center', fontsize=12)
            axes[1,1].set_title('Test Success Rate (N/A)')
        
        plt.tight_layout()
        plt.show()
        
        print("📊 Performance visualizations generated successfully")
        
    except ImportError:
        print("📊 Matplotlib not available - install for visualizations")
    except Exception as e:
        print(f"📊 Visualization error: {e}")

def generate_deployment_report(test_results):
    """Generate deployment readiness report."""
    
    if test_results is None:
        print("📝 No test results available for deployment report")
        return
    
    print("🚀 PMFlow BNN v0.2.0 Deployment Readiness Report")
    print("="*60)
    
    tests = test_results['tests']
    passed_tests = sum(1 for test in tests.values() if test['status'] == 'success')
    total_tests = len(tests)
    success_rate = (passed_tests / total_tests) * 100 if total_tests > 0 else 0
    
    # Deployment readiness assessment
    critical_tests = ['model_creation', 'embarrassingly_parallel']
    critical_passed = sum(1 for test in critical_tests if test in tests and tests[test]['status'] == 'success')
    
    deployment_ready = (success_rate >= 80.0 and critical_passed == len(critical_tests))
    
    print(f"📊 Overall Status: {'✅ READY FOR DEPLOYMENT' if deployment_ready else '⚠️ NEEDS ATTENTION'}")
    print(f"📈 Success Rate: {success_rate:.1f}% ({passed_tests}/{total_tests})")
    print(f"🔧 Critical Tests: {critical_passed}/{len(critical_tests)} passed")
    
    print(f"\n🎯 Next Steps:")
    if deployment_ready:
        print("   1. ✅ Commit library to GitHub")
        print("   2. ✅ Test GitHub import functionality") 
        print("   3. ✅ Deploy to Jetson Nano for edge testing")
        print("   4. ✅ Run cross-system performance validation")
    else:
        print("   1. ❌ Fix failing tests before deployment")
        for test_name, test_data in tests.items():
            if test_data['status'] == 'failed':
                print(f"      - {test_name}: {test_data['error']}")
        print("   2. ❌ Re-run comprehensive tests")
        print("   3. ❌ Achieve >80% success rate before GitHub deployment")
    
    print(f"\n💡 Recommendations:")
    
    # Performance recommendations
    if 'embarrassingly_parallel' in tests and tests['embarrassingly_parallel']['status'] == 'success':
        ep_results = tests['embarrassingly_parallel']['results']
        if ep_results['peak_efficiency'] < 0.8:
            print(f"   ⚡ Consider optimizing parallel efficiency (current peak: {ep_results['peak_efficiency']:.2f}x)")
        else:
            print("   ✅ Excellent parallel scaling achieved")
    
    # Resource recommendations
    system_info = test_results['system_info']
    if system_info['cpu']['cpu_count_logical'] >= 4:
        print("   🖥️ Multi-core system detected - optimal for PMFlow BNN")
    else:
        print("   ⚠️ Limited cores detected - consider CPU optimizations")
    
    if system_info['gpu']['cuda_available']:
        print("   🎮 GPU available - leverage for large-scale testing")
    else:
        print("   💻 CPU-only system - focus on CPU optimizations")
    
    return deployment_ready

# Run analysis if test results are available
if 'test_results' in locals() and test_results is not None:
    print("🔍 Analyzing test results...")
    analyzed_results = analyze_test_results(test_results)
    
    print("\n" + "="*50)
    visualize_performance_data(test_results)
    
    print("\n" + "="*50)
    deployment_ready = generate_deployment_report(test_results)
else:
    print("⏳ Results analysis pending comprehensive test completion")
    print("📝 This analysis will run automatically after comprehensive testing")